# Count eggs using roboflow trained roboflow 3.0

## Import packages

In [4]:
import os
import supervision as sv
from inference import get_roboflow_model
import torch
import onnxruntime as rt
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import multiprocessing as mp

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


## Define functions

In [16]:
def callback(image_slice: np.ndarray) -> sv.Detections:
    results = model.infer(image_slice, confidence=0.45, iou_threshold=0.5)[0] # Reset confidence level here
    return sv.Detections.from_inference(results)

In [ ]:
def count_eggs(image_path):
    # Read image path
    image = cv2.imread(image_path)

    # Decode the QR code in the image
    qcd = cv2.QRCodeDetector()

    retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(image)
    points

    if not retval:
        decoded_info = [os.path.basename(image_path)]
    else:
        # Calculate the center of the square
        center = np.mean(points, axis=1)

        # Expand the points by the specified factors
        scale_x = 1.6
        scale_y = 2.25
        expanded_points = np.copy(points)
        expanded_points[:, :, 0] = scale_x * (points[:, :, 0] - center[:, 0]) + center[:, 0]
        expanded_points[:, :, 1] = scale_y * (points[:, :, 1] - center[:, 1]) + center[:, 1]

        # Convert points to integer coordinates
        expanded_points = expanded_points.astype(int)

        # Create a mask with the same dimensions as the image
        mask = np.ones(image.shape[:2], dtype=np.uint8) * 255

        # Fill the mask with the expanded points
        cv2.fillPoly(mask, expanded_points, 0)

        # Apply the mask to the image to remove the specified points
        image = cv2.bitwise_and(image, image, mask=mask)
    
    # Slice Image and run inference using callback function
    slicer = sv.InferenceSlicer(callback=callback, slice_wh=(640, 640))
    sliced_detections = slicer(image=image)

    # Save output image
    box_annotator = sv.BoxAnnotator()
    # You can also use sv.MaskAnnotator() for instance segmentation models
    # mask_annotator = sv.MaskAnnotator()
    annotated_image = box_annotator.annotate(
        scene=image.copy(), detections=sliced_detections)
    # Convert the annotated image from numpy array to PIL Image
    image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    annotated_image_pil = Image.fromarray(image_rgb)
    annotated_image_resized = annotated_image_pil.resize((4000, 6000))
    annotated_image_resized.save(f'/Users/path/to/output/{decoded_info[0]}.png')

    # Save information to create a pandas df for export
    class_names = sliced_detections['class_name']
    class_id = sliced_detections.class_id
    confidence = sliced_detections.confidence

    # save objects into a single df
    df = (pd.DataFrame({'ID': class_names, 'class_id': class_id, 'confidence': confidence})
        .value_counts(subset = 'ID')
        .to_frame(decoded_info[0])
        .T.assign(total_eggs = len(sliced_detections)))
    
    return df


In [ ]:
# Load images
directory = "/Users/path/to/egg_images"
image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory)]
model = get_roboflow_model(model_id="egg_training-bi/1", api_key="l6XPyOniqM4Ecq129cpf")
print(image_paths)
  

['/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0117.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0116.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0114.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0128.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0129.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0115.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0139.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0111.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0110.JPG', '/Users/aja294/Documents/Trout_local/egg_count/images/input/2024_11_01_egg_images/IMG_0138.JPG', '/Users/aja294/Documents/Trou

2024-11-12 13:00:56.177158 [W:onnxruntime:, coreml_execution_provider.cc:81 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 12 number of nodes in the graph: 233 number of nodes supported by CoreML: 220


In [19]:
# Multi Image run for model
results = []

# Process images sequentially using a for loop
for image_path in image_paths:
    try:
        result = count_eggs(image_path)
        results.append(result)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        continue

In [ ]:
# Combine all DataFrames into a single DataFrame
final_df = pd.concat(results)

# Print the final DataFrame
print(final_df)

ID            egg_eyed  egg_blank  egg_dead  total_eggs
CX-021-2024       4786       1122         8        5916
CX-039-2024       2303        443       505        3251
CX-019-2024       1634        474      1150        3258
IMG_0128.JPG      3130       1044         9        4183
IMG_0129.JPG      2474       1069       917        4460
CX-039-2024       2762        316         5        3083
CX-038-2024       2711        600       661        3972
IMG_0111.JPG      3082       1135         9        4226
CX-036-2024       1687        658      1450        3795
CX-031-2024       1367        760      1567        3694
CX-023-2024       1822        947      1594        4363
IMG_0113.JPG      2585        537         7        3129
CX-028-2024        378        635      2491        3504
CX-030-2024       4541        435        58        5034
CX-022-2024       2242        593       879        3714
CX-037-2024       2700        375       333        3408
CX-034-2024        477        878      1342     

In [ ]:
final_df.to_csv('/Users/path/to/data/output/egg_count_results.csv', index=True)